In [1]:
import requests
from bs4 import BeautifulSoup
import pyarabic.araby as araby
import re
import pandas as pd

In [2]:
def remove_any_non_arabic_char(text):
    non_arabic_char = re.compile('[^\s\\u0600-\u06FF]')
    text_with_no_spaces = re.sub(non_arabic_char, "", text)
    text_with_single_spaces = " ".join(re.split("\s+", text_with_no_spaces))
    return text_with_single_spaces

# Source 1

In [ ]:
URLs ={
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%b9%d8%b1%d8%a7%d9%82%d9%8a%d8%a9/" : "IRQ",
       "https://www.lyricsongation.com/kalemat-aghani-saudiya/" : "GLF",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d9%83%d9%88%d9%8a%d8%aa%d9%8a%d8%a9/" : "GLF",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%a7%d9%85%d8%a7%d8%b1%d8%a7%d8%aa%d9%8a%d8%a9/" : "GLF",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%b9%d9%85%d8%a7%d9%86%d9%8a%d8%a9/" : "GLF",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%a8%d8%ad%d8%b1%d9%8a%d9%86%d9%8a%d8%a9/" : "GLF",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d9%82%d8%b7%d8%b1%d9%8a%d8%a9/" : "GLF",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%ac%d8%b2%d8%a7%d8%a6%d8%b1%d9%8a%d8%a9/" : "NOR",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d9%85%d8%ba%d8%b1%d8%a8%d9%8a%d8%a9/" : "NOR",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%aa%d9%88%d9%86%d8%b3%d9%8a%d8%a9/" : "NOR",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%a7%d8%b1%d8%af%d9%86%d9%8a%d8%a9/": "LEV",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d8%b3%d9%88%d8%b1%d9%8a%d8%a9/" : "LEV",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d9%84%d8%a8%d9%86%d8%a7%d9%86%d9%8a%d8%a9/" : "LEV",
       "https://www.lyricsongation.com/%d9%83%d9%84%d9%85%d8%a7%d8%aa-%d8%a7%d8%ba%d8%a7%d9%86%d9%8a-%d9%81%d9%84%d8%b3%d8%b7%d9%8a%d9%86%d9%8a%d8%a9/" : "LEV",
       }

for url in URLs:    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    musics = soup.find_all(class_='listing listing-blog listing-blog-5 clearfix')[0].find_all(class_='featured clearfix')
    text_list = []
    f = open(f"{URLs[url]}-music.txt", "a" , encoding="utf-8")
    while len(soup.find_all(class_='btn-bs-pagination')) != 0:
        for music in musics:
            for link in music.find_all('a'):
                lyric_page = requests.get(link.get('href'))
                lyric_page_soup = BeautifulSoup(lyric_page.content, "html.parser")
                lyrics = lyric_page_soup.find_all(class_='entry-content clearfix single-post-content')[0].find_all('p')
                for tag in lyrics[1:]:
                    text = araby.strip_diacritics(tag.get_text())
                    text = remove_any_non_arabic_char(text)
                    text_list.append(text)
                    f.write(f"{text}\n")
        page = requests.get(soup.find_all(class_='btn-bs-pagination')[0].get('href'))
        print(soup.find_all(class_='btn-bs-pagination')[0].get('href'))
        soup = BeautifulSoup(page.content, "html.parser")
        try:
            musics = soup.find_all(class_='listing listing-blog listing-blog-5 clearfix')[0].find_all(class_='featured clearfix')
        except:
            print(len(text_list), URLs[url])
            f.close()  

In [ ]:
!pip uninstall twint

In [ ]:
!pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [3]:
import twint
import nest_asyncio
nest_asyncio.apply()

In [ ]:
c = twint.Config()

c.Store_json = True
c.Geo = '33.2232,43.6793,400km' 
c.Output = './tweets/tweets.json'

twint.run.Search(c)

In [6]:
tweeets_df = pd.read_json("./tweets/tweets.json" , lines=True)["tweet"]
tweets = pd.read_json("./tweets/tweets.json" , lines=True)["tweet"].to_list()

In [19]:
tweeets_df = tweeets_df[tweeets_df != " "]

In [20]:
len(tweeets_df)

181960

In [21]:
tweeets_df = tweeets_df.apply(remove_any_non_arabic_char)

In [25]:
for tweet in tweeets_df.sample(30):
    print(tweet)

من النعم الي الله انعم علي فيها اني صرت الاخو الكبير ، وعسى الله يجعلني خير اخ وسند 
 الان چیپسه چقد براتون تموم شد؟؟؟
 الصدق والوفاء 
 نجاح رونالدو مع الفريق هذي ما فيه شك المشكله في المدرب اقل بوايد من انه يكون مدرب للاعبين بهذي الجوده
 اتفق من الصبح
حلببب ام ام الاغنيه و ما زال الحلب مستمر
 ههههههههههههههههه باع مو وجه نعمة والله صوجي هوا هذا الي يثخنها وية الخوات
 رحمه على والديك انت منيلك الاسرار
 كيف الحال
 قال ابن القيم ولأهل الجهاد في هذا من الهداية والكشف ماليس لأهل المجاهدة وقال الأوزاعي وابن المبارك إذا اختلف الناس في شيء فانظروا ما عليه أهل الثغر يعني أهل الجهاد قال تعالى والَّذِينَ جاهدُوا فِينَا لَنَهدِيَنَّهُمْ سُبُلَنَا وإِنَّ اللَّهَ لمع المُحسنِينَ
 سعی میکنم
كـٰٓــؤدخٓـٓصٰٓــمــ نمشي 
 فديتج مروه خوش ترضعين
شسالفه مشاعر الليل مشتغلة من الظهر للان 
 صباح الرضا
 ههههههههههههههههههههههههههههههههههههههههههههههههههههههههههههههه صارت
 از محسن رضایی بپرسین
 خذوني معاكم 
يصير ما يجي يوم الأحد ولا السبوع بكبره يصير نسوي له سكيب ؟ من الحين اني تعبانة 
لا تتحدى جبروتي في التخلي

In [ ]:
for tweet in tweets:
    print(remove_any_non_arabic_char(tweet))